In [1]:
import numpy as np

def arc_point_on_earth(d_km, az_deg, earth_radius_km=6371):
    """
    Compute the coordinates of a point on Earth's surface after traveling an arc
    distance `d_km` from the origin (TX0) in the direction `az_deg` (azimuth),
    taking Earth's curvature into account.

    The output is in a local ENU-like coordinate system where:
        - Origin is at TX0 on Earth's surface.
        - XY plane is tangent to Earth at TX0.
        - Z axis points downward toward Earth's center.

    Parameters:
        d_km: float
            Arc distance to travel along the Earth's surface (in kilometers).
        az_deg: float
            Azimuth angle in degrees (0° = East, 90° = North).
        earth_radius_km: float
            Radius of the Earth (default is 6371 km).

    Returns:
        np.ndarray: A 3D point [x, y, z] in kilometers in the local tangent frame.
    """
    R = earth_radius_km
    az = np.radians(az_deg)
    delta = d_km / R  # central angle in radians

    # Coordinates on the unit sphere, assuming starting point is at (0,0,R)
    x_sphere = np.sin(delta) * np.cos(az)
    y_sphere = np.sin(delta) * np.sin(az)
    z_sphere = np.cos(delta)

    # Convert to local tangent coordinate system at TX0
    x_local = R * x_sphere*1e3
    y_local = R * y_sphere*1e3
    z_local = R * (1 - z_sphere)*1e3  # vertical drop due to Earth's curvature

    return np.array([x_local, y_local, -z_local])  # z points downward

# -------------------------------
# Generate RX positions
# -------------------------------

# Distance list (in km)
# distances_km = [1, 2, 3, 4, 5]
distances_km = [0.5]

# Auto-generate equally spaced azimuths
num_points = len(distances_km)

azimuths_deg = np.linspace(0, 360, num_points, endpoint=False)

# First point at origin (TX0)
gnd_positions = [np.array([0.0, 0.0, 0.0])]

# Generate remaining RX points
for d_km, az in zip(distances_km, azimuths_deg):
    pos = arc_point_on_earth(d_km, az)
    gnd_positions.append(pos)

# Convert to array
gnd_positions = np.array(gnd_positions)

# Print result
for i, pos in enumerate(gnd_positions):
    print(f"TX{i}(m): {pos}")


TX0(m): [0. 0. 0.]
TX1(m): [ 4.99999999e+02  0.00000000e+00 -1.96201537e-02]


In [2]:
def compute_satellite_intersection_point_enu(az_deg, el_deg, sat_orbit_m, tx_pos=None):
    """
    Given azimuth/elevation angles, compute the intersection point with the satellite shell.
    The origin is defined in the local ENU frame centered at the ground transmitter.

    Args:
        az_deg: Azimuth angle in degrees
        el_deg: Elevation angle in degrees
        sat_orbit_m: Satellite orbit height above Earth's surface in meters
        tx_pos: Ground TX position in ECEF coordinates, defaults to [0, 0, earth_radius_m]

    Returns:
        point_enu: np.array([x, y, z]) in meters (ENU coordinates)
        delay_ms: Propagation delay in milliseconds
        distance_m: Propagation distance in meters
    """
    earth_radius_m = 6371e3
    sat_radius_m = earth_radius_m + sat_orbit_m
    c = 3e8  # Speed of light (m/s)

    if tx_pos is None:
        tx_pos = np.array([0, 0, earth_radius_m])  # Default ground TX position

    # Convert angles to radians
    az = np.radians(az_deg)
    el = np.radians(el_deg)

    # Compute ray direction vector (ENU direction)
    d = np.array([
        np.cos(el) * np.cos(az),
        np.cos(el) * np.sin(az),
        np.sin(el)
    ])

    # Solve for intersection with spherical shell
    o = tx_pos
    a = np.dot(d, d)
    b = 2 * np.dot(o, d)
    c_quad = np.dot(o, o) - sat_radius_m**2
    discriminant = b**2 - 4 * a * c_quad

    if discriminant < 0:
        raise ValueError("No intersection with satellite shell")

    t1 = (-b - np.sqrt(discriminant)) / (2 * a)
    t2 = (-b + np.sqrt(discriminant)) / (2 * a)
    t = min(t for t in (t1, t2) if t > 0)

    point_ecef = o + t * d
    point_enu = point_ecef - tx_pos

    distance_m = np.linalg.norm(point_enu)
    delay_ms = distance_m / c * 1e3  # Convert to milliseconds

    return point_enu, delay_ms, distance_m


# custom_tx = np.array([1000.0, 2000.0, 6371e3])
# compute_satellite_intersection_point_enu(az, el, 550e3, tx_pos=custom_tx)


sat_orbit_m = 550e3  # LEO orbit height

# angles = [
#     (45, 90),
#     (120, 80),
    
# ]
angles = [(45, 90), (120, 80), (75,70),(80,70), (65, 85),(130,75)]
sat_positions = []
delays_ms = []
fspl_db = []

frequency_hz = 10e9
wavelength = 3e8 / frequency_hz

for az, el in angles:
    pos, delay, dist = compute_satellite_intersection_point_enu(az, el, sat_orbit_m)
    sat_positions.append(pos)
    delays_ms.append(delay)
    fspl = 20 * np.log10(4 * np.pi * dist / wavelength)
    fspl_db.append(fspl)

# -------------------------------
# Output
# -------------------------------
sat_positions = np.array(sat_positions)
delays_ms = np.array(delays_ms)
fspl_db = np.array(fspl_db)

print("Satellite shell intersection points (ENU) [meters]:\n", sat_positions)
print("\nPropagation delays [ms]:\n", delays_ms)
print("\nFree-space path loss [dB]:\n", fspl_db)


Satellite shell intersection points (ENU) [meters]:
 [[ 2.38137915e-11  2.38137915e-11  5.50000000e+05]
 [-4.84301607e+04  8.38834990e+04  5.49322180e+05]
 [ 5.15413921e+04  1.92355094e+05  5.47134431e+05]
 [ 3.45804104e+04  1.96115253e+05  5.47134431e+05]
 [ 2.03296876e+04  4.35971558e+04  5.49832825e+05]
 [-9.44601913e+04  1.12573272e+05  5.48439683e+05]]

Propagation delays [ms]:
 [1.83333333 1.8593212  1.94082767 1.94082767 1.83977699 1.8926218 ]

Free-space path loss [dB]:
 [167.24902598 167.37128571 167.74393677 167.74393677 167.27950094
 167.52547405]


In [3]:
def compute_az_el_dist(sat_pos, gnd_pos, frequency_hz=None):
    """
    Compute azimuth, elevation, distance, and optionally number of wavelengths
    from TX to satellite in ENU coordinates.

    Args:
        sat_pos: Satellite position in ECEF or local ENU (3,)
        gnd_pos: TX position in same frame as sat_pos (3,)
        frequency_hz: Carrier frequency (Hz). If provided, will return distance in wavelengths.

    Returns:
        az_deg: Azimuth angle in degrees
        el_deg: Elevation angle in degrees
        dist_m: Distance in meters
        n_wavelengths (optional): Distance divided by wavelength (if frequency provided)
    """
    vec = sat_pos - gnd_pos
    dist = np.linalg.norm(vec)

    dx, dy, dz = vec / dist

    el_rad = np.arcsin(dz)
    az_rad = np.arctan2(dy, dx)

    az_deg = (np.degrees(az_rad) + 360) % 360
    el_deg = np.degrees(el_rad)

    if frequency_hz is not None:
        wavelength = 3e8 / frequency_hz
        n_waves = dist / wavelength
        return az_deg, el_deg, dist, n_waves

    return az_deg, el_deg, dist





frequency_hz = 10e9  

for i, tx in enumerate(gnd_positions):
    print(f"\nFrom TX{i}:")
    for j, sat in enumerate(sat_positions):
        az, el, dist, n_waves = compute_az_el_dist(sat, tx, frequency_hz)
        print(f"  SAT{j}: az={az:.2f}°, el={el:.2f}°, dist={dist:.2f} m, λ count ≈ {n_waves:.2f}")





From TX0:
  SAT0: az=45.00°, el=90.00°, dist=550000.00 m, λ count ≈ 18333333.33
  SAT1: az=120.00°, el=80.00°, dist=557796.36 m, λ count ≈ 18593212.01
  SAT2: az=75.00°, el=70.00°, dist=582248.30 m, λ count ≈ 19408276.66
  SAT3: az=80.00°, el=70.00°, dist=582248.30 m, λ count ≈ 19408276.66
  SAT4: az=65.00°, el=85.00°, dist=551933.10 m, λ count ≈ 18397769.90
  SAT5: az=130.00°, el=75.00°, dist=567786.54 m, λ count ≈ 18926218.02

From TX1:
  SAT0: az=180.00°, el=89.95°, dist=550000.25 m, λ count ≈ 18333341.56
  SAT1: az=120.26°, el=79.97°, dist=557840.01 m, λ count ≈ 18594667.14
  SAT2: az=75.14°, el=70.01°, dist=582204.27 m, λ count ≈ 19406809.02
  SAT3: az=80.14°, el=70.01°, dist=582218.84 m, λ count ≈ 19407294.55
  SAT4: az=65.54°, el=85.02°, dist=551914.93 m, λ count ≈ 18397164.20
  SAT5: az=130.15°, el=74.97°, dist=567869.96 m, λ count ≈ 18928998.54


In [4]:
from sionna.rt import Scene, Receiver, Transmitter, PlanarArray, PathSolver
import vsat_dish_3gpp
import numpy as np
import math

jam_rows = 32
jam_cols = 32
jam_antennas = jam_cols*jam_rows

sat_rows = 32
sat_cols = 32
sat_antennas = sat_cols*sat_rows

tx_rows = 32
tx_cols = 32
tx_antennas = tx_cols*tx_rows
# Create scene
scene = Scene()     
for tx_name in scene.transmitters:
    scene.remove(tx_name)
for rx_name in scene.receivers:
    scene.remove(rx_name)   
# Antenna pattern
scene.tx_array = PlanarArray(num_rows=tx_rows ,
                             num_cols=tx_cols,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="tr38901",
                             polarization="V")

scene.rx_array = PlanarArray(num_rows=1,
                             num_cols=1,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="vsat_dish",
                             polarization="V")

# TX at origin
tx = Transmitter(name="tx",
                 position=[0, 0, 550e3],
                 power_dbm=10,
                 display_radius=200)
scene.add(tx)
tx.look_at(gnd_positions[0])
# Define RX positions (550 km above ground, different Y rows)

# Add RXs to scene

for i, pos in enumerate(gnd_positions):
    rx = Receiver(name=f"rx{i}", position=pos, display_radius=200)
    scene.add(rx)
    rx.look_at(tx)


# Let TX point at the center RX (rx1)


# Simulation settings
scene.synthetic_array = True 
scene.frequency = 10e9

# Run propagation solver
p_solver  = PathSolver()
paths = p_solver(scene=scene,
                 max_depth=0,
                 los=True,
                 synthetic_array=True,
                 seed=41)

# CIR analysis
a, tau = paths.cir(normalize_delays=False, out_type="numpy")


c = 3e8  # speed of light (m/s)
fc = 10e9  # Carrier frequency: 10 GHz
wavelength = c / fc
bandwidth = 100e6  # 100 MHz
tx_power_dbm = 50  # transmit power in dBm
noise_figure_db = 5  # dB
k = 1.38e-23  # Boltzmann constant
T = 150  # temperature in Kelvin
noise_power_dbm = 10 * np.log10(k * T * bandwidth) + 30 + noise_figure_db

Tx_power_watt = 10 ** (tx_power_dbm / 10) / 1000  # dBm → Watt
N0_watt = 10 ** (noise_power_dbm / 10) / 1000     # dBm → Watt


In [5]:

# Remove singleton dims: (3, 1, 1, 1, 1, 1) → (3,)
a_squeezed = np.squeeze(a)
h_target = a_squeezed [0,:].reshape(-1, 1)
h_i = a_squeezed[1:, :] 
h_i = h_i.reshape(num_points, -1, 1)
interference_term = np.matmul(h_i, h_i.conj().transpose(0, 2, 1))  
interference_term = np.sum(interference_term, axis=0)   # Sum over all ntn-users


In [6]:
h_i.shape

(1, 1024, 1)

In [7]:
snr_linear = (np.abs(a_squeezed) ** 2) * Tx_power_watt / N0_watt
snr_db = 10 * np.log10(snr_linear)

In [8]:
from BeamformingCalc import svd_bf, nulling_bf

lambda_ = 1e1
w_r = np.array([[1]]) 
w_t = np.ones((1, tx_antennas)) / np.sqrt(tx_antennas)
v_null, _, _,_ = nulling_bf(h_target, w_r, interference_term, lambda_,tx_antennas)

In [9]:
Tx_ground_dBm = 50
Tx_jam_dBm = 50
Tx_ground_watt = 10 ** (Tx_ground_dBm / 10) / 1000  # dBm → Watt
Tx_jam_watt = 10 ** (Tx_jam_dBm / 10) / 1000  # dBm → Watt

In [10]:
SINR = 10 * np.log10((np.abs(w_t @ h_target @ w_r) ** 2) * Tx_ground_watt / ((np.abs(w_t @ h_i @ w_r) ** 2)* Tx_jam_watt+N0_watt) )
print("SINR (dB):", SINR.item())   
SINR_NULL = 10 * np.log10((np.abs(v_null.conj().T @ h_target @ w_r) ** 2) * Tx_ground_watt / ((np.abs(v_null.conj().T @ h_i @ w_r) ** 2)* Tx_jam_watt+N0_watt) )   
print("SINR_NULL (dB):", SINR_NULL.item() )

SINR (dB): 0.003035641171697582
SINR_NULL (dB): 17.90266065809571


In [11]:
INR = 10 * np.log10(((np.abs(w_t @ h_i @ w_r) ** 2)* Tx_jam_watt / N0_watt) )
INR_nulling = 10 * np.log10(((np.abs(v_null.conj().T @ h_i @ w_r) ** 2)* Tx_jam_watt / N0_watt) )
print("INR (dB):", INR.item())   
print("INR_nulling (dB):", INR_nulling.item())   

INR (dB): 52.68519901119414
INR_nulling (dB): 2.0606717826487992


In [12]:
SNR = 10 * np.log10((np.abs(w_t @ h_target @ w_r) **  2) * Tx_ground_watt / (N0_watt) )
print("SNR (dB):", SNR.item())   
SNR_NULL = 10 * np.log10((np.abs(v_null.conj().T @ h_target @ w_r) ** 2) * Tx_ground_watt / (N0_watt) )   
print("SNR_NULL (dB):", SNR_NULL.item())

SNR (dB): 52.688258054918975
SNR_NULL (dB): 22.064387208293414
